In [126]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

In [127]:
iOption0 = 0
is_sigmalised = 1
sim = 0
Min_Entries = 10
N_files = 1
Do_save = 1

In [128]:
isigma = 2
N_layers = 16
N_hists = 4
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/"
file_names="my-m_ee_Run14AuAu_83rd_new_19842_1065runs" 
hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [129]:
if sim :
    file_path="../sim/output/Newembed/dca/" 
    file_names="piminus_50M_v5"

In [130]:
hists_read = []

infile = root.TFile.Open(file_path+file_names+f".root", "read")
for ilayer in range(N_layers):
    hist_select_3D0 = []
    for ihist in range(len(hist_select_3D_names)):
        hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
        hist_select_3D0[-1].SetDirectory(root.nullptr)
    hists_read.append(hist_select_3D0)

infile.Close()

In [131]:
delta_z = 5
z_bins = [i for i in range(9,30,delta_z)]
z_means = array('d', [i-19.5 for i in z_bins])
z_error = array('d', [0.5*delta_z for i in z_bins])
print(z_means, z_error)
pad_limits = [[2,3.7],[-1,0.7]]

array('d', [-10.5, -5.5, -0.5, 4.5, 9.5]) array('d', [2.5, 2.5, 2.5, 2.5, 2.5])


In [132]:
def sdphi_fitter(ihits, sigma=isigma):

    iHist = hists_read[ihits][iOption0+is_sigmalised*2]

    Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg = [], [], [], [], [], [], []

    for iz in z_bins:

        phi_z =  array( 'd' )
        phi_z_err =  array( 'd' )
        Mean_z =  array( 'd' )
        Sigma_z =  array( 'd' )
        Mean_z_err =  array( 'd' )
        Sigma_z_err =  array( 'd' )
        int_fg_local =  array( 'd' )

        N_ybins_loc = iHist.GetNbinsY()
        for iy in range(N_ybins_loc):
            proj_local = iHist.ProjectionX(iHist.GetName()+f"_{iz}_{iy}", iy+1, iy+1, iz+1, iz+1)
            if proj_local.GetEntries() < Min_Entries:
                continue
            
            phi = iHist.GetYaxis().GetBinCenter(iy+1)
            phi_z.append(phi)
            phi_z_err.append(iHist.GetYaxis().GetBinWidth(iy+1)/2.)

            c1 = root.TCanvas("c1"+proj_local.GetName(),"c1"+proj_local.GetName(),720,360)
            gaus = root.TF1("gaus"+proj_local.GetName(), "gaus", -0.0025, 0.0025)
            proj_local.Draw()
            proj_local.Fit(gaus, "Q", "", -0.0025, 0.0025)
            c1.Update()
            if Do_save: c1.SaveAs("output/dynamic_delta_fits/" + proj_local.GetName() + ".png")

            Mean_z.append(gaus.GetParameter(1))
            Sigma_z.append(gaus.GetParameter(2))
            Mean_z_err.append(min(gaus.GetParError(1)*1000, gaus.GetParameter(2)/3))
            Sigma_z_err.append(min(gaus.GetParError(2), 0.05*gaus.GetParameter(2)))


            int_fg_local.append(gaus.Integral(gaus.GetParameter(1)-0.002,gaus.GetParameter(1)+0.002))
            #int_bg_local.append(2*gaus.Integral(gaus.GetParameter(1)+sigma*gaus.GetParameter(2),0.0025))

            #m2_pos.Draw("same") m2_neg.Draw("same")
            #c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )
        Mean_local.append(Mean_z)
        Mean_local_err.append(Mean_z_err)
        Sigma_local.append(Sigma_z)
        Sigma_local_err.append(Sigma_z_err)
        phi_local.append(phi_z)
        phi_local_err.append(phi_z_err)
        int_fg.append(int_fg_local)

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, phi_local, phi_local_err, int_fg]


In [133]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter, [i for i in range(Ntr)])
pool.close()

In [134]:
print(len(output_array[0][0]))

5


In [135]:
Means, Means_err, Sigmas, Sigmas_err, phis, phis_err, int_fg = [], [], [], [], [], [], []
for i in range(N_layers):
    Means.append(output_array[i][0])
    Means_err.append(output_array[i][1])
    Sigmas.append(output_array[i][2])
    Sigmas_err.append(output_array[i][3])
    phis.append(output_array[i][4])
    phis_err.append(output_array[i][5])
    int_fg.append(output_array[i][6])
    
print("Means", len(Means), len(Means[0]), len(Means[0][1]))

Means 16 5 16


In [136]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_0iter = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(2):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_0iter_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.0)
                fit_functions1[-1].SetParLimits(1,0.0001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions_0iter.append(fit_functions0)

In [137]:
g_sigma = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = 0., 0.003
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_sigma_layer = []
    h2.append(Format_Pad_old(pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Sigmas[ilayer])):
        n_phi = len(Sigmas[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Sigmas[ilayer][ized],phis_err[ilayer][ized], Sigmas_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [138]:
fit_mean_functions = []
for ilayer in range(N_layers):
    fit_functions = []
    for ized in range(len(z_bins)):
        fit_functions.append(root.TF1(f"fit_functions_mean_iter_{ized}_"+layers[ilayer],"[0]/10000*sin(x)+[1]/10000*cos(x)+[2]",-1.5,4.5))
        fit_functions[-1].FixParameter(2,0.00)
        #fit_functions[-1].FixParameter(1,0.00)
    fit_mean_functions.append(fit_functions)

In [139]:
g_mean = []
h2=[]
c3 = root.TCanvas(f"c3",f"c3",1400,125*N_layers)
c3.Divide(2,int(N_layers/2),0,0)
Min, Max = -0.002, 0.002
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for ilayer in range(N_layers):
    c3.cd(ilayer+1)
    g_mean_layer = []
    h2.append(Format_Pad_old( pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
    root.gPad.SetTopMargin(0.0)
    root.gPad.SetRightMargin(0.0)
    h2[-1].Draw()
    legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for ized in range(len(Means[ilayer])):
        n_phi = len(Means[ilayer][ized])
        if n_phi<1: continue
        g_sigma_layer.append(root.TGraphErrors(n_phi, phis[ilayer][ized], Means[ilayer][ized],phis_err[ilayer][ized], Means_err[ilayer][ized]))
        Format_Graph(g_sigma_layer[-1],20,1, ized, 3, ized,1,1)
        g_sigma_layer[-1].Fit(fit_mean_functions[ilayer][ized],"Q", "", pad_limits[ilayer%2][0]+0.3, pad_limits[ilayer%2][1])
        g_sigma_layer[-1].Draw("same p")
    g_sigma.append(g_sigma_layer)
    legends[-1].Draw()
        
    
    #g_sigma_f.append(g_sigma_layer)
#g_sigma.append(g_sigma_f)
c3.Draw()
#c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [140]:
for ilayer in range(N_layers):
    for ized in range(len(z_bins)):
        print(f"Layer {ilayer}, z_bin {ized}, dx: {fit_mean_functions[ilayer][ized].GetParameter(0):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(0):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, dy: {fit_mean_functions[ilayer][ized].GetParameter(1):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(1):.6f}")
        print(f"Layer {ilayer}, z_bin {ized}, c: {fit_mean_functions[ilayer][ized].GetParameter(2):.6f} +/- {fit_mean_functions[ilayer][ized].GetParError(2):.6f}")

Layer 0, z_bin 0, dx: 72.530461 +/- 6.706923
Layer 0, z_bin 0, dy: 23.478074 +/- 1.719399
Layer 0, z_bin 0, c: 0.000000 +/- 0.000000
Layer 0, z_bin 1, dx: 24.537807 +/- 2.689252
Layer 0, z_bin 1, dy: 7.678579 +/- 1.055560
Layer 0, z_bin 1, c: 0.000000 +/- 0.000000
Layer 0, z_bin 2, dx: 3.093832 +/- 2.658485
Layer 0, z_bin 2, dy: -1.722282 +/- 0.940398
Layer 0, z_bin 2, c: 0.000000 +/- 0.000000
Layer 0, z_bin 3, dx: -3.325605 +/- 2.601597
Layer 0, z_bin 3, dy: -4.007289 +/- 0.977189
Layer 0, z_bin 3, c: 0.000000 +/- 0.000000
Layer 0, z_bin 4, dx: -19.950358 +/- 2.456598
Layer 0, z_bin 4, dy: -5.211173 +/- 0.916896
Layer 0, z_bin 4, c: 0.000000 +/- 0.000000
Layer 1, z_bin 0, dx: -6.916020 +/- 2.917300
Layer 1, z_bin 0, dy: -4.338723 +/- 1.082826
Layer 1, z_bin 0, c: 0.000000 +/- 0.000000
Layer 1, z_bin 1, dx: 3.628945 +/- 3.285807
Layer 1, z_bin 1, dy: -7.380219 +/- 1.439367
Layer 1, z_bin 1, c: 0.000000 +/- 0.000000
Layer 1, z_bin 2, dx: 4.845062 +/- 2.830748
Layer 1, z_bin 2, dy: -1.05

In [141]:
# Example for plotting means in 3D for ilayer=0
ilayer = 2
g3d = root.TGraph2D()
point = 0
for ized, z in enumerate(z_bins):
    n_phi = len(Means[ilayer][ized])
    for iphi in range(n_phi):
        phi = phis[ilayer][ized][iphi]
        mean = Means[ilayer][ized][iphi]
        g3d.SetPoint(point, phi, z_means[ized], mean)
        point += 1
# Define the fit function: ([0]+[1]*zed)*sin(phi)+([2]+[3]*zed)*cos(phi)+[4]
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", -2, 4, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these

c3d = root.TCanvas("c3d", "3D Mean", 800, 600)


g3d.SetTitle("Mean vs #phi and z;#phi;z;Mean")
g3d.SetMarkerStyle(20)
g3d.SetMarkerColor(root.kBlue)
# Fit the 3D graph
g3d.Fit(fit_func, "Q")  # "Q" for quiet

# Draw the points and the fit surface
g3d.Draw("P0 surf1")
fit_func.Draw("same surf1")

c3d.Draw()

In [142]:
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 7.4 ± 16
Parameter [1]: -11 ± 2.3
Parameter [2]: -1e+02 ± 81
Parameter [3]: 0.1 ± 0.95
Parameter [4]: -9.6 ± 7.5


In [143]:
# Fill a TH3D with (phi, z, mean) and fit 2D slices with the given function

layers3D = [0,2]
# Define binning
n_bins_phi = 20
n_bins_z = len(z_means)
phi_min, phi_max = pad_limits[ilayer%2][0], pad_limits[ilayer%2][1]
z_min, z_max = min(z_means), max(z_means)
mean_min, mean_max = -0.002, 0.002

th3 = root.TH3D("th3", "Mean vs #phi and z;#phi;z;Mean", n_bins_phi, pad_limits[ilayer%2][0], pad_limits[ilayer%2][1], n_bins_z, z_min, z_max, 100, mean_min, mean_max)

# Fill TH3D
for ilayer in layers3D:
    for ized, z in enumerate(z_bins):
        n_phi = len(Means[ilayer][ized])
        for iphi in range(n_phi):
            phi = phis[ilayer][ized][iphi]
            mean = Means[ilayer][ized][iphi]
            mean_err = int_fg[ilayer][ized][iphi]**0.5
            th3.Fill(phi, z_means[ized], mean, int_fg[ilayer][ized][iphi])
            #print(th3.FindBin(phi, z_means[ized], mean))
            ibin = th3.FindBin(phi, z_means[ized], mean)
            th3.SetBinError(ibin, th3.GetBinContent(ibin)**0.5)  # Set the error as the square root of the content

c1 = root.TCanvas("c1", "3D Histogram", 800, 600)
th3.Draw("LEGO2")  # Draw the 3D histogram
fit_func = root.TF2("fit_func", "([0]+[1]*y)/10000*sin(x)+([2]+[3]*y)/10000*cos(x)+[4]/1000", phi_min, phi_max, min(z_means), max(z_means))
fit_func.SetParameters(0, 0, 0, 0, 0)  # You can tune these
fit_func.FixParameter(4, 0)  # Fix the constant term to zero for the fit
th3.Fit(fit_func, "Q")  # Fit the 3D histogram
fit_func.Draw("same surf1")
c1.Draw()

# fit_results now contains the fit parameters for each z slice

In [144]:
#print(f"chi2/ndf = {fit_func.GetChisquare():.2f} / {fit_func.GetNDF()} = {fit_func.GetChisquare()/fit_func.GetNDF():.2f}")
# Print the fit parameters and their errors for the 3D fit function
for i in range(5):
    par = fit_func.GetParameter(i)
    err = fit_func.GetParError(i)
    print(f"Parameter [{i}]: {par:.2g} ± {err:.2g}")

Parameter [0]: 4.3 ± 1.4
Parameter [1]: -2.4 ± 0.37
Parameter [2]: -0.57 ± 0.44
Parameter [3]: -1.7 ± 0.12
Parameter [4]: 0 ± 0


In [145]:
#print(fit_func.Eval(3.14, 0))  # Example evaluation at phi=3.0, z=20
# Print the equation for dphi(z, phi) with obtained fit parameters
print(
    f"dphi = ({fit_func.GetParameter(0):.6g} + {fit_func.GetParameter(1):.6g}*z)/10000 * sin(phi) + "
    f"({fit_func.GetParameter(2):.6g} + {fit_func.GetParameter(3):.6g}*z)/10000 * cos(phi) + "
    f"{fit_func.GetParameter(4)/1000:.6g}"
)
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000")

dphi = (4.29152 + -2.40276*z)/10000 * sin(phi) + (-0.566794 + -1.72733*z)/10000 * cos(phi) + 0
x + (4.29 + -2.4*z)/10000
y + (-0.567 + -1.73*z)/10000


In [146]:
print(f"x + ({fit_func.GetParameter(0):.3g} + {fit_func.GetParameter(1):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*sin(phi)")
print(f"y + ({fit_func.GetParameter(2):.3g} + {fit_func.GetParameter(3):.3g}*z)/10000 + r * sin( {fit_func.GetParameter(4):.3g}/1000)*cos(phi)")

x + (4.29 + -2.4*z)/10000 + r * sin( 0/1000)*sin(phi)
y + (-0.567 + -1.73*z)/10000 + r * sin( 0/1000)*cos(phi)


## Formula for Alignment Fit

The function used for fitting the mean shift as a function of azimuthal angle (φ) and longitudinal position (z) is:

<div align="center">

Δφ(z, φ) = ([0] + [1]·z)/10000 · sin(φ) + ([2] + [3]·z)/10000 · cos(φ) + [4]/1000

</div>

**where:**
- [0], [1], [2], [3], [4] are fit parameters extracted from the data,
- z is the longitudinal coordinate (e.g., along the beam axis),
- φ is the azimuthal angle.

---

### Why use this function for a 3D histogram fit?

- **Physical Motivation:**  
  Misalignments of detector layers often manifest as sinusoidal modulations in the measured hit positions as a function of φ, with possible linear dependence on z due to tilts or twists.

- **Parameter Extraction:**  
  Fitting this function to the 3D histogram of mean residuals (mean shift vs. φ and z) allows extraction of alignment parameters:
    - The coefficients of sin(φ) and cos(φ) (and their z-dependence) correspond to translations and tilts of the detector layer in the transverse plane.
    - The constant term can represent a global offset.

- **Usage:**  
  After fitting, the obtained parameters can be used to correct (align) the detector geometry, improving tracking and vertexing performance.